# Convolutional Neural Network

Construccion, afinamiento, entrenamiento y pruebas de una red neuronal convolucional clasificadora de images.

### Librearias

In [1]:
# Librerias de TensorFlow y Keras para construccion de la red neuronal
import tensorflow as tf
from keras import regularizers
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam

# Librerias para procesamiento y aumento de datos de imágenes
from keras.preprocessing.image import ImageDataGenerator

# Librerias para afinamiento de hiperparametros
import keras_tuner as kt

# Librerias para manejo de rutas y operaciones del sistema de archivos
import os

### Data Augmentation

In [2]:
# Resolucion objetivo de las imagenes
# 100x100 pixeles
# 3 canales de color (RGB)
target_size = (100, 100)

In [3]:
data_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 30,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

**Generador para imagenes de entrenamiento**

In [4]:
training_data = os.path.join(os.getcwd(), 'dataset/split/train')

train_generator = data_generator.flow_from_directory(
    training_data,
    target_size = target_size,
    batch_size = 32,
    class_mode = 'categorical',
    color_mode = 'grayscale'
)

train_batch_qty = len(train_generator)
print(f"Number of batches: {train_batch_qty}")

Found 210 images belonging to 3 classes.


Number of batches: 7


**Generador para imagenes de testeo**

In [5]:
testing_data = os.path.join(os.getcwd(), 'dataset/split/test')

test_generator = data_generator.flow_from_directory(
    testing_data,
    target_size = target_size,
    batch_size = 32,
    class_mode = 'categorical',
    color_mode = 'grayscale'
)

test_batch_qty = len(test_generator)
print(f"Number of batches: {test_batch_qty}")

Found 60 images belonging to 3 classes.
Number of batches: 2


**Generador para imagenes de validacion**

In [6]:
validation_data = os.path.join(os.getcwd(), 'dataset/split/val')

valid_generator = data_generator.flow_from_directory(
    validation_data,
    target_size = target_size,
    batch_size = 32,
    class_mode = 'categorical',
    color_mode = 'grayscale'
)

val_batch_qty = len(valid_generator)
print(f"Number of batches: {val_batch_qty}")

Found 30 images belonging to 3 classes.
Number of batches: 1


## Modelado de la Red Neronal

### Construccion del Modelo

In [7]:
# Cantidad de clases
num_classes = len(train_generator.class_indices)
print(f"Number of classes: {num_classes}")

Number of classes: 3


In [8]:
def build_model(hp):
    model = Sequential()
    
    # Primera capa de Input convolucional (fija)
    model.add(Conv2D(
        filters=hp.Int('input_filter', min_value = 8, max_value = 256, step = 8),
        kernel_size=hp.Choice('input_kernel', values=[3, 5]),
        kernel_regularizer=regularizers.l2(hp.Float('input_l2_regularizer', 1e-5, 1e-1, sampling='log')),
        padding='same',
        activation='relu',
        input_shape=(100, 100, 1)
    ))
    model.add(BatchNormalization())
    
    # Capas variables de convolucional, batch normalization, max pooling y dropout
    for i in range(hp.Int('num_conv_layers', 1, 6)):
        model.add(Conv2D(
            filters=hp.Int(f'conv_{i}_filter', min_value = 8, max_value = 256, step = 8),
            kernel_size=hp.Choice(f'conv_{i}_kernel', values=[3, 5]),
            kernel_regularizer=regularizers.l2(hp.Float('l2_regularizer', 1e-5, 1e-1, sampling='log')),
            padding='same',
            activation='relu'
        ))
        # Hiperparámetro booleano para BatchNormalization
        if hp.Boolean(f'batch_norm_{i}'):
            model.add(BatchNormalization())
            
        # Hiperparámetro booleano para MaxPooling2D
        if hp.Boolean(f'pooling_{i}'):
            model.add(MaxPooling2D(pool_size=(2, 2)))

        # Hiperparámetro booleano para Dropout
        if hp.Boolean(f'dropout_{i}'):
            model.add(Dropout(hp.Float(f'dropout_rate_{i}', 0.1, 0.7, step=0.1)))
    

    # Capas de aplanamiento (fija)
    model.add(Flatten())
  
    # Capa densa (fijas)
    model.add(Dense(num_classes, activation='softmax'))

    # Selección del optimizador y su tasa de aprendizaje como hiperparámetros
    optimizer_choice = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('adam_learning_rate', values=[1e-2, 1e-3, 1e-4])
        )
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(
            learning_rate=hp.Choice('rmsprop_learning_rate', values=[1e-2, 1e-3, 1e-4])
        )
    elif optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=hp.Choice('sgd_learning_rate', values=[1e-2, 1e-3, 1e-4])
        )

    # Compilación del modelo
    model.compile(
        optimizer = optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

### Tuning del modelo con Keras Tuner

In [9]:
# Crear los afinadores de hiperparametros

# Afinador de busqueda aleatoria
random_tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=50,
    executions_per_trial=1,
    directory='tuner',
    project_name='random_tuner'
)

# Afinador de Hyperband
hyper_tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='tuner',
    project_name='hyper_tuner'
)

# Afinador de busqueda bayesiana
bayes_tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=50,
    executions_per_trial=1,
    directory='tuner',
    project_name='bayes_tuner'
)


**Callback para optimizacion de busqueda de hiperparametros**

In [10]:
# Callbacks para detener el entrenamiento si no hay mejora despues de 5 epocas
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [11]:
# Buscar los mejores hiperparametros con el afinador de busqueda aleatoria
random_tuner.search(train_generator, epochs=50, validation_data = valid_generator, callbacks=[stop_early])

# Obtener los mejores hiperparametros encontrados por el afinador de busqueda aleatoria
best_random_hps = random_tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 50 Complete [00h 00m 28s]
val_accuracy: 0.36666667461395264

Best val_accuracy So Far: 0.6333333253860474
Total elapsed time: 00h 24m 03s


In [12]:
# Buscar los mejores hiperparametros con el afinador de Hyperband
hyper_tuner.search(train_generator, epochs=50, validation_data = valid_generator, callbacks=[stop_early])
# Obtener los mejores hiperparametros encontrados por el afinador de Hyperband
best_hyper_hps = hyper_tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 90 Complete [00h 00m 17s]
val_accuracy: 0.3333333432674408

Best val_accuracy So Far: 0.5666666626930237
Total elapsed time: 00h 34m 30s


In [13]:
# Buscar los mejores hiperparametros con el afinador de busqueda bayesiana
bayes_tuner.search(train_generator, epochs=50, validation_data = valid_generator, callbacks=[stop_early])
# Obtener los mejores hiperparametros encontrados por el afinador de busqueda bayesiana
best_bayes_hps = bayes_tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 50 Complete [00h 00m 56s]
val_accuracy: 0.46666666865348816

Best val_accuracy So Far: 0.5333333611488342
Total elapsed time: 00h 32m 38s


#### Resumen del Modelo ideal encontrado

In [14]:
random_model = random_tuner.hypermodel.build(best_random_hps)
random_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 100, 100, 8)       208       
                                                                 
 batch_normalization_2 (Batc  (None, 100, 100, 8)      32        
 hNormalization)                                                 
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 100, 128)     25728     
                                                                 
 conv2d_4 (Conv2D)           (None, 100, 100, 16)      51216     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 16)       0         
 )                                                               
                                                                 
 conv2d_5 (Conv2D)           (None, 50, 50, 128)      

In [15]:
hyper_model = hyper_tuner.hypermodel.build(best_hyper_hps)
hyper_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 100, 100, 16)      160       
                                                                 
 batch_normalization_3 (Batc  (None, 100, 100, 16)     64        
 hNormalization)                                                 
                                                                 
 conv2d_7 (Conv2D)           (None, 100, 100, 128)     18560     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 50, 50, 128)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 50, 50, 96)        110688    
                                                                 
 batch_normalization_4 (Batc  (None, 50, 50, 96)      

In [16]:
bayes_model = bayes_tuner.hypermodel.build(best_bayes_hps)
bayes_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 100, 100, 192)     4992      
                                                                 
 batch_normalization_5 (Batc  (None, 100, 100, 192)    768       
 hNormalization)                                                 
                                                                 
 conv2d_13 (Conv2D)          (None, 100, 100, 208)     359632    
                                                                 
 batch_normalization_6 (Batc  (None, 100, 100, 208)    832       
 hNormalization)                                                 
                                                                 
 flatten_3 (Flatten)         (None, 2080000)           0         
                                                                 
 dense_3 (Dense)             (None, 3)                

### Entrenamiento de modelo optimizado

In [17]:
random_model.fit(train_generator, epochs=100, validation_data=valid_generator)

Epoch 1/100
7/7 [==============================] - 1s 74ms/step - loss: 3.6029 - accuracy: 0.3524 - val_loss: 3.5483 - val_accuracy: 0.3333
Epoch 2/100
7/7 [==============================] - 0s 57ms/step - loss: 3.5845 - accuracy: 0.3286 - val_loss: 3.5033 - val_accuracy: 0.3333
Epoch 3/100
7/7 [==============================] - 0s 60ms/step - loss: 3.4942 - accuracy: 0.4238 - val_loss: 3.4554 - val_accuracy: 0.3000
Epoch 4/100
7/7 [==============================] - 0s 60ms/step - loss: 3.4325 - accuracy: 0.4000 - val_loss: 3.4055 - val_accuracy: 0.3333
Epoch 5/100
7/7 [==============================] - 0s 60ms/step - loss: 3.3729 - accuracy: 0.3810 - val_loss: 3.3542 - val_accuracy: 0.4000
Epoch 6/100
7/7 [==============================] - 0s 60ms/step - loss: 3.3172 - accuracy: 0.4143 - val_loss: 3.3032 - val_accuracy: 0.3000
Epoch 7/100
7/7 [==============================] - 0s 60ms/step - loss: 3.2648 - accuracy: 0.4381 - val_loss: 3.2510 - val_accuracy: 0.3333
Epoch 8/100
7/7 [===

In [18]:
hyper_model.fit(train_generator, epochs=100, validation_data=valid_generator)

Epoch 1/100
7/7 [==============================] - 1s 66ms/step - loss: 8.2991 - accuracy: 0.2810 - val_loss: 8.2376 - val_accuracy: 0.3667
Epoch 2/100
7/7 [==============================] - 0s 45ms/step - loss: 8.2660 - accuracy: 0.2810 - val_loss: 8.2371 - val_accuracy: 0.3667
Epoch 3/100
7/7 [==============================] - 0s 53ms/step - loss: 8.2615 - accuracy: 0.2905 - val_loss: 8.2375 - val_accuracy: 0.2333
Epoch 4/100
7/7 [==============================] - 0s 48ms/step - loss: 8.2467 - accuracy: 0.2905 - val_loss: 8.2367 - val_accuracy: 0.3000
Epoch 5/100
7/7 [==============================] - 0s 51ms/step - loss: 8.2620 - accuracy: 0.2952 - val_loss: 8.2373 - val_accuracy: 0.2333
Epoch 6/100
7/7 [==============================] - 0s 52ms/step - loss: 8.2589 - accuracy: 0.3000 - val_loss: 8.2363 - val_accuracy: 0.2000
Epoch 7/100
7/7 [==============================] - 0s 49ms/step - loss: 8.2221 - accuracy: 0.4000 - val_loss: 8.2362 - val_accuracy: 0.2667
Epoch 8/100
7/7 [===

In [19]:
bayes_model.fit(train_generator, epochs=100, validation_data=valid_generator)

Epoch 1/100
7/7 [==============================] - 1s 136ms/step - loss: 2190.2180 - accuracy: 0.3524 - val_loss: 11567.2412 - val_accuracy: 0.3000
Epoch 2/100
7/7 [==============================] - 1s 119ms/step - loss: 2431.7002 - accuracy: 0.3619 - val_loss: 29331.5410 - val_accuracy: 0.3333
Epoch 3/100
7/7 [==============================] - 1s 125ms/step - loss: 2662.9062 - accuracy: 0.4000 - val_loss: 11554.3916 - val_accuracy: 0.3333
Epoch 4/100
7/7 [==============================] - 1s 130ms/step - loss: 2645.9116 - accuracy: 0.3952 - val_loss: 5603.0781 - val_accuracy: 0.2333
Epoch 5/100
7/7 [==============================] - 1s 121ms/step - loss: 2078.2742 - accuracy: 0.4429 - val_loss: 9760.2627 - val_accuracy: 0.3333
Epoch 6/100
7/7 [==============================] - 1s 152ms/step - loss: 1783.7815 - accuracy: 0.4381 - val_loss: 6693.3276 - val_accuracy: 0.3667
Epoch 7/100
7/7 [==============================] - 1s 135ms/step - loss: 2222.3970 - accuracy: 0.3667 - val_loss: 2

### Evaluacion del Modelo Final

In [20]:
random_eval_result = random_model.evaluate(test_generator)
for metric, value in zip(random_model.metrics_names, random_eval_result):
    print(f'{metric}: {value}')

2/2 [==============================] - 0s 355ms/step - loss: 1.5897 - accuracy: 0.4500
loss: 1.5896732807159424
accuracy: 0.44999998807907104


In [21]:
hyper_eval_result = hyper_model.evaluate(test_generator)
for metric, value in zip(hyper_model.metrics_names, hyper_eval_result):
    print(f'{metric}: {value}')

2/2 [==============================] - 0s 372ms/step - loss: 8.1829 - accuracy: 0.4000
loss: 8.182923316955566
accuracy: 0.4000000059604645


In [22]:
bayes_eval_result = bayes_model.evaluate(test_generator)
for metric, value in zip(bayes_model.metrics_names, bayes_eval_result):
    print(f'{metric}: {value}')

2/2 [==============================] - 1s 1s/step - loss: 87.4278 - accuracy: 0.3333
loss: 87.42784118652344
accuracy: 0.3333333432674408
